In [6]:
import numpy as np

In [7]:
from typing import Callable
from numpy import array, dot, ndarray, zeros

class Layer:
    w: ndarray
    b: ndarray
    act: Callable[[ndarray], ndarray]
    def __init__(self, w: ndarray, b: ndarray | None = None, act: Callable[[ndarray], ndarray] = lambda x: x) -> None:
        self.w = w
        if b is None:
            self.b = zeros(w.shape[0])
        else:
            self.b = b
        self.act = act

class NeuNet:
    layers: list[Layer]
    def __init__(self) -> None:
        self.layers = []

    def add_layer(self, layer: Layer | tuple[str, int]) -> None:
        if type(layer) is Layer:
            if len(self.layers) != 0 and self.layers[-1].w.shape[0] != layer.w.shape[-1]:
                raise ValueError(f"dimentions of subsequent layers does not match up: {self.layers[-1].w.shape} and {layer.w.shape}")
            self.layers.append(layer)
            return
        raise NotImplementedError("Other types are not supported yet")


    def feed_forward(self, inputs: ndarray) -> ndarray:
        result = inputs
        if len(self.layers) == 0 or self.layers[-1].w.shape[0] != 1:
            raise ValueError("Invalid output layer")
        for layer in self.layers:
            result = layer.w @ result
            # print(f"after weights: {result}")
            result =  result + layer.b
            # print(f"after bias: {result}")
            result = layer.act(result)
            # print(f"after activation: {result}")
        return result


In [8]:
from numpy import array, vectorize

act = lambda x: 0. if x < 0.5 else 1.

act = vectorize(act)

hidden = Layer(array([[0.3, 0.3, 0], [0.4, -0.5, 1]]), act=act)
output = Layer(array([[-1, 1]]), act=act)
nn = NeuNet()
nn.add_layer(hidden)
nn.add_layer(output)
nn.feed_forward(array([0, 0, 1]))

array([1.])

In [9]:
x_test = [
    [0., 0., 1.],
    [1., 1., 0.],
    [1., 1., 1.],
    [0., 0., 0.],
    [1., 0., 0.],
    [0., 1., 0.]
    ]
y_test = [
    [1.],
    [1.],
    [1.],
    [0.],
    [0.],
    [0.]
]

In [10]:
for x, y in zip(x_test, y_test):
    r = nn.feed_forward(array(x))
    for rr, yr in zip(r, y):
        if (rr<0.5) != (yr<0.5):
            break
    else:
        continue
    print(f"Output on {x} is different from expected: actual={r} != reference={y}")

Output on [1.0, 1.0, 0.0] is different from expected: actual=[0.] != reference=[1.0]
Output on [1.0, 1.0, 1.0] is different from expected: actual=[0.] != reference=[1.0]
